In [1]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup

In [18]:
from distutils.log import error


limit = 8 # setting to 3 articles limit per page

def getDate(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    dates = soup.find_all(class_ = "date") 
    dateArray=[]
    count = 1
    for date in dates :
        dateArray.append(date.get_text().split()[1:])
        if count >= limit :
            break
        count += 1    
    return dateArray


# tweak start and end for faster compile
def getStartEnd(url,startnum,endnum):
    try :
        check = True
        start = getDate("{}{}".format(url,startnum))
        while check :
            if (start[2][0] == 'Aug' and start[2][2] == '2022') :
                check = False
                break
            startnum += 1
            start = getDate("{}{}".format(url,startnum))
            print(start)
        print(startnum)
        # coronavirus special - oldest date to mar 2020
        if url == 'https://www.investing.com/news/coronavirus/' :
            endnum = 650 # last updated 17 October 2022
        else :
            check = True
            end = getDate("{}{}".format(url,endnum))
            while check :
                if end[2][0] == 'Jan' and end[2][2] == '2018' :
                    check = False
                    break
                endnum += 1
                end = getDate("{}{}".format(url,endnum))    
                print(end)
            print(endnum)
        return (startnum,endnum)
    except : 
        print("error")

def getArticles(url,startnum,endnum):
    df = pd.DataFrame(columns=['Date','Title'])
    start,end = getStartEnd(url,startnum,endnum)
    while start <= end :
        page = requests.get("{}{}".format(url,start))
        soup = BeautifulSoup(page.content, 'html.parser')
        dates = soup.find_all(class_="date")
        titles = soup.find_all(class_ = "title") 
        for i in range(limit) :
            df_new = pd.DataFrame({"Date" : dates[i].text[3:], "Title" : titles[i].text},index=[1])
            df = pd.concat([df,df_new],ignore_index=True)
        print(start, end, "[Page {}/{}]".format(start, end))
        start += 1
    return df

In [20]:
df_tsla = getArticles("https://www.investing.com/equities/tesla-motors-news/",36,811)
print(df_tsla.head())
print(df_tsla.shape)
df_tsla.to_csv('../Data/News/Stock/tsla_news.csv',index=False)

error


TypeError: cannot unpack non-iterable NoneType object

In [7]:
df_aapl = getArticles("https://www.investing.com/equities/apple-computer-inc-news/",30,1034) 
print(df_aapl.head())
print(df_aapl.shape)
df_aapl.to_csv('../Data/News/Stock/aapl_news.csv',index=False)

error


TypeError: cannot unpack non-iterable NoneType object

In [8]:
df_meta = getArticles("https://www.investing.com/equities/facebook-inc-news/",20,1211) 
print(df_meta.head())
print(df_meta.shape)
df_meta.to_csv('../Data/News/Stock/meta_news.csv',index=False)

error


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
df_world = getArticles("https://www.investing.com/news/world-news/",98,4313)
print(df_world.head())
print(df_world.shape)
df_world.to_csv('../Data/News/Global/world_news.csv',index=False)

In [ ]:
df_politics = getArticles("https://www.investing.com/news/politics/",9,780) 
print(df_politics.head())
print(df_politics.shape)
df_politics.to_csv('../Data/News/Global/politics_news.csv',index=False)

In [ ]:
df_coronavirus = getArticles("https://www.investing.com/news/coronavirus/",3,650) 
print(df_coronavirus.head())
print(df_coronavirus.shape)
df_coronavirus.to_csv('../Data/News/Global/coronavirus_news.csv',index=False)